In [ ]:
!pip install transformers tokenizers sentencepiece tiktoken


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import pandas as pd
import torch
import optuna
import wandb

from torch import nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn.functional import softmax as torch_softmax
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_linear_schedule_with_warmup
import numpy as np

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# PRETRAIN_MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
PRETRAIN_MODEL = "microsoft/deberta-v3-base"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Data

The dataset used in this project is sourced from [Kaggle - Coronavirus tweets NLP](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification/data).

In [5]:
# Load the dataset
df_train = pd.read_csv("data/Corona_NLP_train.csv", encoding='latin1')
df_test = pd.read_csv("data/Corona_NLP_test.csv", encoding='latin1')

# Display first few rows
df_test.head()

,ï»¿UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02/03/2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02/03/2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02/03/2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02/03/2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03/03/2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [6]:
# map sentiment to integers
sentiment_map = {
    'Extremely Negative': 0,
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3,
    'Extremely Positive': 4
}

df_train['Label'] = df_train['Sentiment'].map(sentiment_map)
df_test['Label'] = df_test['Sentiment'].map(sentiment_map)

df_test.head()

,ï»¿UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Label
0,1,44953,NYC,02/03/2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0
1,2,44954,"Seattle, WA",02/03/2020,When I couldn't find hand sanitizer at Fred Me...,Positive,3
2,3,44955,NaN,02/03/2020,Find out how you can protect yourself and love...,Extremely Positive,4
3,4,44956,Chicagoland,02/03/2020,#Panic buying hits #NewYork City as anxious sh...,Negative,1
4,5,44957,"Melbourne, Victoria",03/03/2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,2


#### Split the training data into train and eval

In [ ]:
train_df, eval_df = train_test_split(
    df_train,
    test_size=0.3,
    random_state=42,
    stratify=df_train['Label']
)

train_df = train_df[['OriginalTweet', 'Label']].reset_index(drop=True)
eval_df = eval_df[['OriginalTweet', 'Label']].reset_index(drop=True)
test_df = df_test[['OriginalTweet', 'Label']].reset_index(drop=True)

train_df.to_csv('data/train_data.csv', index=False)
eval_df.to_csv('data/eval_data.csv', index=False)

### Look at the model

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(PRETRAIN_MODEL, num_labels=5, ignore_mismatched_sizes=True).to(device)
#model # Lets just look at the structure of the reoerta model from HF

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Check how the data is tokenized

In [17]:
# check num of tokens to choose truncation max_length
#tok = AutoTokenizer.from_pretrained(PRETRAIN_MODEL, use_fast=True)
#lens = df_train['OriginalTweet'].astype(str).map(lambda t: len(tok.encode(t, add_special_tokens=True)))
#lens.describe(), lens.quantile([0.90, 0.95, 0.99]).to_dict()

from transformers import DebertaV2Tokenizer
tok = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=True)
lens = df_train['OriginalTweet'].astype(str).map(
    lambda t: len(tok.encode(t, add_special_tokens=True))
)
print(lens.describe(), lens.quantile([0.90, 0.95, 0.99]).to_dict())

count    41157.000000
mean        55.744078
std         21.730697
min          3.000000
25%         40.000000
50%         55.000000
75%         70.000000
max        249.000000
Name: OriginalTweet, dtype: float64 {0.9: 84.0, 0.95: 92.0, 0.99: 107.0}


In [8]:
## Setup & Train

In [14]:
class TweetDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.texts = dataframe['OriginalTweet'].tolist()
        self.labels = dataframe['Label'].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [15]:
def early_stop_check(patience, best_f1, best_f1_epoch, current_f1, current_f1_epoch):
    early_stop_flag = False
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_f1_epoch = current_f1_epoch
    else:
        if current_f1_epoch - best_f1_epoch > patience:
            early_stop_flag = True
    return best_f1, best_f1_epoch, early_stop_flag

In [16]:
def train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs, patience, trial, scheduler):
    best_val_f1 = 0.0
    best_f1_epoch = 0
    early_stop_flag = False
    best_model_state = None

    # Enable automatic mixed precision on CUDA for stability/speed
    scaler = torch.amp.GradScaler("cuda", enabled=(device.type == "cuda"))

    for epoch in range(1, epochs + 1):
        model.train() # Enable training mode
        train_loss = 0.0
        total_train_samples = 0
        correct_train_predictions = 0

        for batch in train_loader: #Iterates over the train_loader, which is a DataLoader object containing batches of training data.
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad(set_to_none=True) # Reset gradients

            # Forward pass (with AMP); save the logits (the raw output of the model) and calculate loss
            with torch.amp.autocast("cuda", enabled=(device.type == "cuda")):
                outputs = model(input_ids, attention_mask=attention_mask) # Forward pass
                logits = outputs.logits
                loss = criterion(logits, labels) # Calculate loss

            # Backward pass (with AMP) + gradient clipping, then update weights using the optimizer
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # NEW: prevent exploding gradients
            scaler.step(optimizer)
            scaler.update()

            # NEW: step the LR scheduler once per optimizer step
            if scheduler is not None:
                scheduler.step()

            # Accumulate training loss and predictions
            train_loss += loss.item() * input_ids.size(0)
            total_train_samples += input_ids.size(0)
            correct_train_predictions += (logits.argmax(dim=1) == labels).sum().item()

        train_loss /= total_train_samples
        train_accuracy = correct_train_predictions / total_train_samples

        ###  Validation loop  ###
        model.eval() # Enable evaluation mode
        val_loss = 0.0
        total_val_samples = 0
        correct_val_predictions = 0

        all_val_labels = []
        all_val_preds = []
        all_val_probs = []

        with torch.no_grad(): # Disable gradient computation
            for batch in val_loader: # iterate on the val_loader's batches 
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = criterion(logits, labels)

                val_loss += loss.item() * input_ids.size(0)
                total_val_samples += input_ids.size(0)
                correct_val_predictions += (logits.argmax(dim=1) == labels).sum().item()

                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(logits.argmax(dim=1).cpu().numpy())
                all_val_probs.append(torch_softmax(logits, dim=1).cpu().numpy())

        # calculate metrics 
        val_loss /= total_val_samples
        val_accuracy = correct_val_predictions / total_val_samples
        val_precision = precision_score(all_val_labels, all_val_preds, average='macro')
        val_recall = recall_score(all_val_labels, all_val_preds, average='macro')
        val_f1 = f1_score(all_val_labels, all_val_preds, average='macro')

        probs = np.concatenate(all_val_probs, axis=0)
        labels_np = np.asarray(all_val_labels)
        try:
            val_auc = roc_auc_score(labels_np, probs, multi_class='ovr', average='macro')
        except ValueError:
            # 
            val_auc = float('nan')

        # Check for early stopping (UNCHANGED: still based on accuracy)
        best_val_f1, best_f1_epoch, early_stop_flag = early_stop_check(
            patience, best_val_f1, best_f1_epoch, val_f1, epoch
        )

        # Save the best model under the best_model_state parameter by f1
        if val_f1 >= best_val_f1:
            best_model_state = model.state_dict()

        print(f"Epoch {epoch}: train_loss={train_loss:.4f} val_f1={val_f1:.4f}, best_val_f1={best_val_f1:.4f}")

        # Log metrics to Weights & Biases - THIS IS WHERE WE TRACK THE RESULTS AND THE PROCESS
        wandb.log({ #log == logging of the training process (e.g. results) - will be done each epoch
            "Epoch": epoch,
            "Train Loss": train_loss,
            "Train Accuracy": train_accuracy,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_accuracy,
            "Validation Precision": val_precision,
            "Validation Recall": val_recall,
            "Validation F1": val_f1,
            "Validation AUC": val_auc,

        })

        if early_stop_flag:  # Checks whether the early stopping condition has been met, as indicated by the early_stop_flag
            break # Exits the training loop immediately if the early stopping condition is satisfied

    if best_model_state is not None: # Save the best model as a .pt file
        torch.save(best_model_state, f"best_model_trial_{trial.number}.pt")

    return best_val_f1

In [20]:

from transformers import DebertaV2Tokenizer
tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=True)
# tokenizer = AutoTokenizer.from_pretrained(PRETRAIN_MODEL, use_fast=True)

# Objective Function for Optuna
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
    weight_decay = trial.suggest_loguniform("weight_decay", 5e-3, 3e-2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])  # use grad accumulation if VRAM is tight
    num_layers = trial.suggest_categorical("num_layers", [2, 3, 4, 5])
    patience = 5

    train_dataset = TweetDataset(train_df, tokenizer) # Create the TweetDataset object
    val_dataset = TweetDataset(eval_df, tokenizer)    # Create the TweetDataset object

    data_collator = DataCollatorWithPadding(tokenizer, padding=True, return_tensors="pt", pad_to_multiple_of=8)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                num_workers=6, pin_memory=True, persistent_workers=True, collate_fn=data_collator)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=6,
                                pin_memory=True, persistent_workers=True, collate_fn=data_collator)

    #model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=5, ignore_mismatched_sizes=True).to(device)
    model = AutoModelForSequenceClassification.from_pretrained(PRETRAIN_MODEL, num_labels=5, ignore_mismatched_sizes=True).to(device)

    # model.base_model.<...> when changing the model to something else
    for param in model.base_model.parameters():    # Freeze layers
        param.requires_grad = False
    for param in model.base_model.encoder.layer[-num_layers:].parameters():     # unfreeze the last "num_layers" of the encoder
        param.requires_grad = True
    for param in model.classifier.parameters():    #unfreeze the classifier
        param.requires_grad = True

    # Define optimizer and loss function
    # AdamW optimizer (transformer-friendly) instead of plain Adam
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # class-weighted CrossEntropy to handle label imbalance
    counts = train_df['Label'].value_counts().sort_index().values
    weights = torch.tensor((counts.sum() / (counts + 1e-9)), dtype=torch.float32, device=device)
    criterion = nn.CrossEntropyLoss(weight=weights)

    # LR scheduler with warmup (linear decay)
    epochs = 20  # keep your epoch budget here so we can compute total steps
    num_training_steps = epochs * len(train_loader)
    num_warmup_steps = int(0.06 * num_training_steps)  # ~6% warmup
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

    # Initialize Weights & Biases - the values in the config are the properties of each trial.
    model_name = PRETRAIN_MODEL.split("/")[1]
    wandb.init(
        # project="bertweet-covid-sentiment",
        project=f"covid-sentiment-twitter-{model_name}",
               config={ 
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "patience": patience,
        "batch_size": batch_size,
        "num_layers": num_layers,
        # "architecture": "BERTweet",
        # "architecture": "RoBERTa",
        "architecture": "DeBERTa",
        "dataset": "COVID-19 NLP"}, 
        name=f"trial_{trial.number}") # The name that will be saved in the W&B platform

    # Train the model and get the best validation accuracy
    best_val_f1 = train_model_with_hyperparams(
        model, train_loader, val_loader, optimizer, criterion,
        epochs=epochs, patience=patience, trial=trial, scheduler=scheduler
    )

    wandb.finish() # Finish the Weights & Biases run
    
    return best_val_f1 # Return best validation acc as the objective to maximize

In [21]:
# Optuna Study
study = optuna.create_study(direction="maximize")  # Specifies that the goal of the optimization is to maximize the objective function
study.optimize(objective, n_trials=15)

[I 2025-08-18 20:44:38,480] A new study created in memory with name: no-name-33d28cfc-8f3e-46c7-bba6-03d3d7801a49
/tmp/ipykernel_2912973/2845401944.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
/tmp/ipykernel_2912973/2845401944.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 5e-3, 3e-2)
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should prob

/home/idok/git/deep-tweet-covid/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 1: train_loss=1.6100 val_f1=0.1352, best_val_f1=0.1352
Epoch 2: train_loss=1.3806 val_f1=0.5088, best_val_f1=0.5088
Epoch 3: train_loss=1.0212 val_f1=0.6071, best_val_f1=0.6071
Epoch 4: train_loss=0.9078 val_f1=0.5771, best_val_f1=0.6071
Epoch 5: train_loss=0.8311 val_f1=0.6415, best_val_f1=0.6415
Epoch 6: train_loss=0.7712 val_f1=0.6603, best_val_f1=0.6603
Epoch 7: train_loss=0.7155 val_f1=0.6754, best_val_f1=0.6754
Epoch 8: train_loss=0.6944 val_f1=0.6713, best_val_f1=0.6754
Epoch 9: train_loss=0.6645 val_f1=0.6818, best_val_f1=0.6818
Epoch 10: train_loss=0.6401 val_f1=0.6767, best_val_f1=0.6818
Epoch 11: train_loss=0.6302 val_f1=0.6814, best_val_f1=0.6818
Epoch 12: train_loss=0.5968 val_f1=0.6877, best_val_f1=0.6877
Epoch 13: train_loss=0.5604 val_f1=0.6855, best_val_f1=0.6877
Epoch 14: train_loss=0.5386 val_f1=0.6927, best_val_f1=0.6927
Epoch 15: train_loss=0.5132 val_f1=0.6898, best_val_f1=0.6927
Epoch 16: train_loss=0.4968 val_f1=0.6843, best_val_f1=0.6927
Epoch 17: train_l

Epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
Train Accuracy,▁▃▅▆▆▆▇▇▇▇▇▇████████
Train Loss,█▇▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
Validation AUC,▁▆▇▇████████████████
Validation Accuracy,▁▅▆▆▇▇██████████████
Validation F1,▁▆▇▇▇███████████████
Validation Loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂
Validation Precision,▁▅▇▇▇▇██████████████
Validation Recall,▁▆▇▇▇███████████████
Epoch,20
Train Accuracy,0.80614


[I 2025-08-18 23:17:57,649] Trial 0 finished with value: 0.6945423274562295 and parameters: {'learning_rate': 3.3114161071670855e-05, 'weight_decay': 0.02579334100215903, 'batch_size': 64, 'num_layers': 5}. Best is trial 0 with value: 0.6945423274562295.
/tmp/ipykernel_2912973/2845401944.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
/tmp/ipykernel_2912973/2845401944.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 5e-3, 3e-2)
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/de

...

[I 2025-08-19 09:13:01,582] Trial 13 finished with value: 0.711288999794248 and parameters: {'learning_rate': 3.75008962541288e-05, 'weight_decay': 0.005042121877492635, 'batch_size': 32, 'num_layers': 5}. Best is trial 10 with value: 0.7378896406857568.
/tmp/ipykernel_2912973/2845401944.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5)
/tmp/ipykernel_2912973/2845401944.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 5e-3, 3e-2)
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/de

/home/idok/git/deep-tweet-covid/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 1: train_loss=1.6070 val_f1=0.1938, best_val_f1=0.1938
Epoch 2: train_loss=1.1848 val_f1=0.5972, best_val_f1=0.5972
Epoch 3: train_loss=0.9125 val_f1=0.6139, best_val_f1=0.6139
Epoch 4: train_loss=0.7977 val_f1=0.6488, best_val_f1=0.6488
Epoch 5: train_loss=0.7276 val_f1=0.6818, best_val_f1=0.6818
Epoch 6: train_loss=0.6773 val_f1=0.6842, best_val_f1=0.6842
Epoch 7: train_loss=0.6404 val_f1=0.6968, best_val_f1=0.6968
Epoch 8: train_loss=0.5982 val_f1=0.6924, best_val_f1=0.6968
Epoch 9: train_loss=0.5572 val_f1=0.6936, best_val_f1=0.6968
Epoch 10: train_loss=0.5295 val_f1=0.7075, best_val_f1=0.7075
Epoch 11: train_loss=0.4894 val_f1=0.7015, best_val_f1=0.7075
Epoch 12: train_loss=0.4621 val_f1=0.6915, best_val_f1=0.7075
Epoch 13: train_loss=0.4342 val_f1=0.7026, best_val_f1=0.7075
Epoch 14: train_loss=0.4038 val_f1=0.7024, best_val_f1=0.7075
Epoch 15: train_loss=0.3856 val_f1=0.7039, best_val_f1=0.7075
Epoch 16: train_loss=0.3640 val_f1=0.7059, best_val_f1=0.7075


Epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
Train Accuracy,▁▄▅▆▆▆▇▇▇▇▇█████
Train Loss,█▆▄▃▃▃▃▂▂▂▂▂▁▁▁▁
Validation AUC,▁▇▇█████████████
Validation Accuracy,▁▆▇▇████████████
Validation F1,▁▆▇▇████████████
Validation Loss,█▃▂▁▁▁▁▁▁▁▁▂▂▂▂▃
Validation Precision,▁▇▇▇████████████
Validation Recall,▁▆▇▇████████████
Epoch,16
Train Accuracy,0.84932


[I 2025-08-19 09:52:27,481] Trial 14 finished with value: 0.7075298468661098 and parameters: {'learning_rate': 2.8145246238112575e-05, 'weight_decay': 0.0065469647902776165, 'batch_size': 32, 'num_layers': 5}. Best is trial 10 with value: 0.7378896406857568.


# Testing - note didn't run yet

In [ ]:
# Function to evaluate the model
def evaluate_model(model_path, test_loader):
    # Load the model
    model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=5, ignore_mismatched_sizes=True).to(device)
    model.load_state_dict(torch.load(model_path)) # loading the trained model
    model = model.to(device)
    model.eval() # eval mode

    all_labels = []
    all_preds = []

    #same idea... just testing and getting resuts...
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = logits.argmax(dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    return {"Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1}

In [ ]:
# Load the test data set
test_dataset = DataLoader(test_df, AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", use_fast=True))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Test multiple models
model_paths = ["best_model_trial_0.pt"]  # Replace with actual model paths
for model_path in model_paths:
    metrics = evaluate_model(model_path, test_loader)
    print(f"Metrics for {model_path}:")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")